In [2]:
import sys
import glob

import numpy as np
import pandas as pd


In [10]:
def get_traj(file):
	df = pd.read_csv(file, names=('tag', 'ts', 'v1', 'v2', 'v3', 'v4'))

	df_s = df[df.tag == 'pos']
	df_s = df_s.rename(columns={'v1' : 'x', 'v2' : 'y', 'v3' : 'z', 'v4' : 'a'})


	df_u = df[df.tag == 'input']
	df_u = df_u.rename(columns={'v1' : 'roll', 'v2' : 'pitch', 'v3' : 'yaw', 'v4' : 'throttle'})

	x, y, z, a = [df_s[v].to_numpy() for v in ['x', 'y', 'z', 'a']]
	roll, pitch, yaw, throttle = [df_u[v].to_numpy() for v in ['roll', 'pitch', 'yaw', 'throttle']]
	
	s = np.stack([x, y, z, a]).T
	u = np.stack([roll, pitch, yaw, throttle]).T


	return (s, u)

In [14]:
logs_folder = '../logs/ident/filt'
logs = []
for log in glob.glob(f'{logs_folder}/*.log'):
	logs.append(get_traj(log))


In [17]:
a = [len(s) for s, u in logs]
print(a, sum(a))

[5289, 5273, 4035, 3521, 5123, 4591] 27832
